In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster


In [38]:
# read in CSV from webscrape
cummins_df = pd.read_csv('/home/nhac/Documents/NSS/Python/capstone/cummins_scrape_1.csv')
cummins_df.head()

In [89]:
for i in range(1, 4):
    for j in range(2, 5):
        print('zone' + str(i) + 'zone' + str(j))

zone1zone2
zone1zone3
zone1zone4
zone2zone2
zone2zone3
zone2zone4
zone3zone2
zone3zone3
zone3zone4


In [97]:
empty_list = []
for i in range(3, 10):
    for j in ['Apple','Peach','Plum','Pear','Nectarine','Cherry','Apricot']:       
        globals()[j + str(i)] = []
        

In [ ]:
# test for results calling column and copying 
poop = cummins_df[['Apple3']].copy()
poop

In [111]:
fruit_list = pd.DataFrame()     # create empty dataframe to append results
for i in range(3,10):     # iterate through numbers to assign for zone
    for j in ['Apple','Peach','Plum','Pear','Nectarine','Cherry','Apricot']:     # for every fruit type names
        col_name = j + str(i)     # concat zone number to fruit type name to get matching column name in cummins_df (Apple3, Apple4, etc.)
        tmp_f_list = cummins_df[[col_name]].copy()     # tmp list with column name matching column name (variable)
        tmp_f_list = tmp_f_list[tmp_f_list[col_name].notna()]     # drop the NaNs from the list for later list concat
        tmp_f_list = tmp_f_list.rename(columns={col_name:'fruit_name'})     # rename the variable column name, with static name
        tmp_f_list['zone'] = i     # add a column to the tmp DF and populate the column with variable RANGE number
        tmp_f_list[['type']] = j     # add a column to the tmp DF and populate with variable fruit type name
        fruit_list = pd.concat([fruit_list, tmp_f_list])     # append results from tmp_f_list to previously empty df, fruit_list

In [114]:
type(fruit_list)

pandas.core.frame.DataFrame

In [115]:
fruit_list.to_csv()

,fruit_name,zone,type
0,Almata,3,Apple
1,Antonovka (Common),3,Apple
2,Black Oxford,3,Apple
3,Blue Pearmain,3,Apple
4,Centennial Crab,3,Apple
...,...,...,...
4,Snow Giant,9,Peach
5,Sugar Giant,9,Peach
6,Sugar May,9,Peach
0,Fantasia,9,Nectarine


In [20]:
geo_h_zones = gpd.read_file('../data/us-hardiness-zones_1291.geojson')

In [21]:
geo_h_zones = geo_h_zones.rename(columns={'name':'zone'})

In [22]:
geo_h_zones = geo_h_zones.to_crs(epsg=4326)

In [23]:
type(geo_h_zones)

geopandas.geodataframe.GeoDataFrame

In [24]:
centroid_df = geo_h_zones.geometry.centroid.to_frame(name='centroid')

/tmp/ipykernel_164604/460883729.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid_df = geo_h_zones.geometry.centroid.to_frame(name='centroid')


In [25]:
geo_df = pd.concat([geo_h_zones, centroid_df] , axis=1)

In [26]:
print(type(geo_df))
geo_df

<class 'geopandas.geodataframe.GeoDataFrame'>


,zone,geometry,centroid
0,2,"POLYGON Z ((-97.24092 49.01284 0.00000, -97.10...",POINT (-94.19724 48.47422)
1,2,"POLYGON Z ((-106.31249 40.49058 0.00000, -106....",POINT (-106.29115 39.56151)
2,2,"POLYGON Z ((-74.82668 44.62830 0.00000, -75.06...",POINT (-74.74326 44.28086)
3,2,"POLYGON Z ((-111.05274 44.70431 0.00000, -111....",POINT (-110.84491 44.39016)
4,2,"POLYGON Z ((-69.82155 46.70390 0.00000, -69.51...",POINT (-68.91906 46.92776)
5,3,"POLYGON Z ((-111.98457 48.96959 0.00000, -112....",POINT (-101.92520 47.78009)
6,4,"POLYGON Z ((-116.08464 49.01875 0.00000, -116....",POINT (-102.10183 44.75713)
7,4,"MULTIPOLYGON Z (((-68.25855 47.37753 0.00000, ...",POINT (-71.75391 44.92231)
8,5,"POLYGON Z ((-80.55821 41.96125 0.00000, -80.49...",POINT (-74.44953 42.81991)
9,5,"MULTIPOLYGON Z (((-87.79937 44.97507 0.00000, ...",POINT (-99.84062 41.80543)


In [27]:
m = folium.Map(location=(31.51073, -96.4247), zoom_start=5, tiles='CartoDB positron')


In [28]:
for _, r in geo_df.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    
    folium.Popup(r['zone']).add_to(geo_j)
    geo_j.add_to(m)


for _, r in geo_df.iterrows():
    lat = r['centroid'].y
    lon = r['centroid'].x
    folium.Marker(location=[lat, lon]),
                   
m